## Assignment 3, PROBLEM 1

Consider the data `X` and `y`, in the cell below. `X` denotes $20$ points in $\mathbb{R}^2$ and `y` corresponds to the labels for these points, i.e. it is a classification problem.

In [1]:
import numpy as np
X = np.array([[0.14774693918368506,0.8537253157278155],[-0.1755517430286779,0.8979710703337818],[0.5227216475286975,0.7448281947022451],[-0.5071170511153492,0.8002027400836075],[-0.39436968212400453,1.0177689414422981],[-0.3983065780966649,1.0443663197782966],[-0.08652771617599643,0.48036820824519255],[0.15352541170101042,0.6820807981911706],[-0.3303348532791869,1.120673883903539],[-0.2656220857139274,0.8526638282828739],[0.7259603693529442,0.25428467532034965],[0.4577253912481767,-0.2358809079980879],[0.9722462145222105,0.13128550836973255],[0.4089349951770505,-0.09503914544452634],[0.9718156747909192,0.3524307824261209],[1.2009353774940565,-0.25004126389987974],[1.271791635779178,-0.07571928320750206],[0.36784476124502913,-0.23743021661715671],[0.8918396050420891,-0.1029336332277948],[0.4501578013678095,-0.13188266835015783]])+np.array([10,0]).reshape(1,-1)
y = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0])

## 1, 2)

Implement the function `perceptron` by filling in `XXX`. Use your implemented `perceptron` function to compute a vector (numpy array) $\hat w$ with shape `(3,1)` such that 
$$
    (\hat w \cdot \hat x_i) l_i > 0, \quad \forall i=1,\ldots,20
$$
put your answer in `hat_w` below (the last dimension is the bias dimension, i.e. the added dimension we used to derive the perceptron)

### Answer

The perceptron algorithm tries to find a linear separator, i.e. a plane in $\mathbb{R}$ that separates the two classes. The task is thus to find $w$ and $t$ such that for the training data $S$, the data consists of pairs $(x_i,y_i)$ the $x_i$ represents our features and the $y_i$ our labels or target.

$$
\begin{aligned}
    w \cdot x_i > t \quad \text{for each $x_i$ labeled $+1$} \\
    w \cdot x_i < t \quad \text{for each $x_i$ labeled $-1$}
\end{aligned}
$$

Adding a new coordinate to our space allows us to consider $\hat x_i = (x_i,1)$ and $\hat w = (w,t)$, this allows us to rewrite the inequalities above as

$$
    (\hat w \cdot \hat x_i) y_i > 0.
$$

#### The algorithm
1. $\hat w = 0$
2. while there exists $\hat x_i$ with $\hat x_i y_i \cdot \hat w \leq 0$, update $\hat w := \hat w+\hat x_i y_i$

In [2]:
def perceptron(X_in,labels,max_iter=10000):
    '''Runs the perceptron algorithm on X_in, labels, and does a maximum of max_iter iterations'''
    
    n_points = len(X_in)
    w = np.array([0,0,0]) # w^ = 0
    X = np.concatenate([X_in,np.ones((n_points,1))],axis=1) # Adding the extra dimension with value 1
    
    j = 0
    while (j < max_iter):
        i = j % n_points # Reset index every n iterations
        if (X[i,:]@w * labels[i] <= 0): # X[i,:] is just the current vector or the iteration
            w = w + X[i,:] * labels[i]
        j += 1
    return w

hat_w = perceptron(X,y)
hat_w

array([-1.49468903, 34.08611021,  3.        ])

## 3)

**Use the vector $\hat w$ that you just found and compute $r = \max_i |x_i|$ (put your result in `r`), finally use this to give an upper bound to the number of iterations needed for the perceptron algorithm to converge on this dataset, see chapter 8 in the ITDS notes. Put the result in `iteration_bound`.**

### Theorem: Perceptron for linearly separable data
> If there exists $w^\ast$ such that $w^\ast \cdot x_i l_i \geq 1$ for all $i$. Then the perceptron algorithm find a $w$ satisfying $w \cdot x_i l_i \geq 0$ for all $i$ in at most $r^2|w^\ast|^2$ updates, where $r = \max_i |x_i|$.

So this theorem guarantees that if the two classes can be separated then the preceptron will also finds a separator in finite time.

### Answer

$$r = \max_i |\hat x_i|$$
$$\min_{margin} = \min_i \hat w \cdot \hat x_i \cdot y_i $$
$$w^* = \frac{\hat w}{\displaystyle \min_{margin}}$$

In [3]:
'''X_hat is just X with an appended 1 in every vector'''
X_hat = np.concatenate([X,np.ones((len(X),1))],axis=1) # Adding the extra dimension with value 1

'''To find r, loop through X_hat, norm every vector and save the one with the biggest norm'''
max_ = 0
for i in X_hat:
    norm = np.linalg.norm(i)
    if (norm > max_):
        max_ = norm
r = max_

'''Compute minimum margin = min(w^*x^*y)'''
min_margin = np.min((X_hat@hat_w).reshape(-1)*y)

w_star = hat_w/min_margin
iteration_bound = np.linalg.norm(w_star)**2*r**2
iteration_bound

78150.30109461867

## Assignment 3, PROBLEM 2

For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. If you download the updated `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [4]:
import pandas as pd
import sklearn as skl

df_train = pd.read_csv('data/indoor_train.csv')

'''Both X and Y needs to be numpied to get the right datatype '''
Xtrain = df_train[['Position X', ' Position Y', 'Position Z']].to_numpy()
Ytrain = df_train['Location'].to_numpy()

from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc_train = svc.fit(Xtrain,Ytrain)

## Assignment 3, PROBLEM 3

## SMS spam filtering [8p]

In the following problem we will explore SMS spam texts. The dataset is the `SMS Spam Collection Dataset` and we have provided for you a way to load the data. If you run the appropriate cell below, the result will be in the `spam_no_spam` variable. The result is a `list` of `tuples` with the first position in the tuple being the SMS text and the second being a flag `0 = not spam` and `1 = spam`.

### 1)

Let $X$ be the random variable that represents each SMS text (an entry in the list), and let $Y$ represent whether text is spam or not i.e. $Y \in \{0,1\}$. Thus $\mathbb{P}(Y = 1)$ is the probability that we get a spam. The goal is to estimate:
$$
    \mathbb{P}(Y = 1 | \text{"free" or "prize" is in } X) \enspace .
$$
That is, the probability that the SMS is spam given that "free" or "prize" occurs in the SMS. (This is precision)
Hint: it is good to remove the upper/lower case of words so that we can also find "Free" and "Prize"; this can be done with `text.lower()` if `text` a string.

### Answer
We model this as a conditional probability where `A` is the outcome `Y=1`, and `B` is the outcome `"free" or "prize" is in X`. We then get that 

$$P(A|B) = \frac{A \cap B}{B}$$

In [5]:
# Run this cell to get the SMS text data
def load_sms():
    import csv
    lines = []
    hamspam = {'ham': 0, 'spam': 1}
    with open('data/spam.csv', mode='r',encoding='latin-1') as f:
        reader = csv.reader(f)
        header = next(reader)
        lines = [[line[1],hamspam[line[0]]] for line in reader]
        
    return lines

spam_no_spam = load_sms()

'''Turn everything to lower case'''
for tup in spam_no_spam:
    text = tup[0]
    tup[0] = text.lower()
    
countA = 0 # Y = 1
countB = 0 # 'free' or 'prize' in X 
countAB = 0

'''Count all the necessary events'''
for tup in spam_no_spam:
    aTrue = False
    bTrue = False
    '''Important: add split to avoid counting words like "freestyle"'''
    if ("prize" in tup[0].split(' ')) or ("free" in tup[0].split(' ')):
        countB += 1
        bTrue = True
    if (tup[1] == 1):
        countA += 1
        aTrue = True
    if (aTrue and bTrue):
        countAB += 1

'''Calculate probabilities'''
tot = len(spam_no_spam)
probA = countA/tot
probB = countB/tot
probAB = countAB/tot

problem4_hatP = probAB/probB

### 2)

Estimate the probability that the word "free" or "prize" is in the text given that it is spam. (This is recall) I.e. estimate
$$
    \mathbb{P}(\text{"free" or "prize" is in } X \mid Y = 1) \enspace .
$$

### Answer

$$P(B|A) = \frac{B \cap A}{A}$$

In [6]:
problem4_hatP2 = probAB/probA

### 3)
Provide a "90\%" interval of confidence around the true probability from **part 1**. I.e. use the Hoeffding inequality to obtain for your estimate $\hat P$. Find $l > 0$ such that the following holds:
$$
    \mathbb{P}(\hat P - l \leq \mathbb{E}[\hat P] \leq \hat P + l) \geq 0.9 \enspace .
$$

### Answer

**Lemma 3.8.** *[Estimating p in Bernoulli] Let $X_1, \ldots, X_n \stackrel{IID}{\sim}$ Bernoulli$(p)$. Then for $\alpha \in (0,1)$ we have for $\delta = \frac{1}{\sqrt{n}} \sqrt{\frac{1}{2}ln \left( \frac{2}{a} \right) }$*

$$\mathbb{P}(\overline{X}_n - \delta \leq p \leq \overline{X}_n + \delta) \geq 1 - \alpha$$

We have that $\alpha = 0.1$, and $n$ is the frequency of event $B$, which is the event that `"free" or "prize" is in X`.

In [7]:
problem4_l = (1/np.sqrt(countB))*np.sqrt((1/2)*np.log(2/0.1))
problem4_l

0.07740455120409898